# Data Loading and Storage数据载入、存储及文件格式

# 1. Reading and Writing Data in Text Format(读写文本格式的数据)
pandas提供了一些用于将表格型数据读取为DataFrame对象的函数，其中read_csv和read_table可能今后用得最多的。这些函数的选项可以划分为以下几个大类：  
- Indexing（索引）：将一个或多个列当做返回的DataFrame处理， 以及是否从文件、 用户获取列名；
- Type inference and data conversion（类型推断和数据转换）：包括用户定义值的转换、 和自定义的缺失值标记列表等；
- Datetime parsing（日期解析）：包括组合功能， 比如将分散在多个列中的日期时间信息组合成结果中的单个列；
- Iterating（迭代）：支持对大文件进行逐块迭代；
- Unclean data issues（不规整数据问题）：跳过一些行、 页脚、 注释或其他一些不重要的东西（ 比如由成千上万个逗号隔开的数值数

In [1]:
import pandas as pd
import numpy as np
!type examples\ex1.csv

Active code page: 65001
a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


In [2]:
df = pd.read_csv('examples/ex1.csv')

In [3]:
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [4]:
pd.read_table('examples/ex1.csv', sep=',')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [3]:
!type examples\ex2.csv  

Active code page: 65001
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


In [6]:
pd.read_csv('examples/ex2.csv', header=None) # 分配默认的列名

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [7]:
pd.read_csv('examples/ex2.csv', names=['a', 'b', 'c', 'd', 'message']) # 自己定义列名

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [8]:
names = ['a', 'b', 'c', 'd', 'message']

In [9]:
# 将message列做成索引
pd.read_csv('examples/ex2.csv', names=names, index_col='message') 

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


In [10]:
!type examples\csv_mindex.csv

Active code page: 65001
key1,key2,value1,value2
one,a,1,2
one,b,3,4
one,c,5,6
one,d,7,8
two,a,9,10
two,b,11,12
two,c,13,14
two,d,15,16


In [11]:
# 层次化索引
parsed = pd.read_csv('examples/csv_mindex.csv', index_col=['key1', 'key2'])
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

In [12]:
list(open('examples/ex3.txt')) # 非固定分隔符分隔，空白格数量大小不等

['            A         B         C\n',
 'aaa -0.264438 -1.026059 -0.619500\n',
 'bbb  0.927272  0.302904 -0.032399\n',
 'ccc -0.264273 -0.386314 -0.217601\n',
 'ddd -0.871858 -0.348382  1.100491\n']

In [13]:
# 传递正则表达式 \s+
result = pd.read_table('examples/ex3.txt', sep='\s+') 

In [14]:
result # 列比行少，自动推断第一列为索引

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


In [4]:
!type examples\ex4.csv 

Active code page: 65001
# hey!
a,b,c,d,message
# just wanted to make things more difficult for you
# who reads CSV files with computers, anyway?
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


In [16]:
# 跳过某些行
pd.read_csv('examples/ex4.csv', skiprows=[0, 2, 3])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [17]:
!type examples\ex5.csv

Active code page: 65001
something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo


In [18]:
result = pd.read_csv('examples/ex5.csv')

In [19]:
result # 缺失数据

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [20]:
pd.isnull(result)

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


In [21]:
# na_values可以用一个列表或集合的字符串表示缺失值
result = pd.read_csv('examples/ex5.csv', na_values=['NULL'])
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [22]:
# 字典的各列可以使用不同的NA标记值
sentinels = {'message': ['foo', 'NA'], 'something': ['two']}

In [23]:
pd.read_csv('examples/ex5.csv', na_values=sentinels)

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


## 逐块读取文本文件

In [24]:
pd.options.display.max_rows = 10 # 显示的更紧凑

In [25]:
res = pd.read_csv('examples/ex6.csv')

In [26]:
res

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
...,...,...,...,...,...
9995,2.311896,-0.417070,-1.409599,-0.515821,L
9996,-0.479893,-0.650419,0.745152,-0.646038,E
9997,0.523331,0.787112,0.486066,1.093156,K
9998,-0.362559,0.598894,-1.843201,0.887292,G


In [27]:
# 制定读取几行
pd.read_csv('examples/ex6.csv', nrows=5)

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


In [28]:
# 要逐块读取文件，可以指定chunksize( 每一块的行数)
chunker = pd.read_csv('examples/ex6.csv', chunksize=1000)

In [29]:
chunker
# 返回的对象可以根据chunksize对文件进行逐块迭代

In [30]:
# 遍历ex6.csv，并对'key'列聚合获得计数值
chunker = pd.read_csv('examples/ex6.csv', chunksize=1000)
tot = pd.Series([])
for piece in chunker:
#     print(type(piece['key'].value_counts()))
    tot = tot.add(piece['key'].value_counts(), fill_value=0)
    
tot = tot.sort_values(ascending=False)

In [31]:
tot[:10]

E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
M    338.0
J    337.0
F    335.0
K    334.0
H    330.0
dtype: float64

## 将数据写出到文本格式

In [32]:
data = pd.read_csv('examples/ex5.csv')
data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [33]:
data.to_csv('examples/out.csv') # 写到以逗号分隔的文件中

In [34]:
!type examples\out.csv

Active code page: 65001
,something,a,b,c,d,message
0,one,1,2,3.0,4,
1,two,5,6,,8,world
2,three,9,10,11.0,12,foo


In [35]:
# 还可以使用其他分隔符
import sys
data.to_csv(sys.stdout, sep='|') # 输出到标准输出

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


In [36]:
data.to_csv(sys.stdout, na_rep='NULL') # 缺失值表示为NULL

,something,a,b,c,d,message
0,one,1,2,3.0,4,NULL
1,two,5,6,NULL,8,world
2,three,9,10,11.0,12,foo


In [37]:
data.to_csv(sys.stdout, index=False, header=False) # 禁用行和列标签

one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo


In [38]:
data.to_csv(sys.stdout, index=False, columns=['a', 'b', 'c']) # 只显示部分列， 并可以指定顺序

a,b,c
1,2,3.0
5,6,
9,10,11.0


In [39]:
# Series也有to_csv方法
dates = pd.date_range('6/28/2019', periods=7)

In [40]:
ts = pd.Series(np.arange(7), index=dates)

In [41]:
ts.to_csv('examples/tseries.csv')

In [42]:
!type examples\tseries.csv 

Active code page: 65001
2019-06-28,0
2019-06-29,1
2019-06-30,2
2019-07-01,3
2019-07-02,4
2019-07-03,5
2019-07-04,6


## 处理分隔符格式

In [43]:
!type examples\ex7.csv

Active code page: 65001
"a","b","c"
"1","2","3"
"1","2","3"


In [44]:
# 对于任何单字符分隔符文件，可以直接用python内置的csv模块
# 将任意已打开的文件或文件型对象传给csv.reader
import csv
f = open('examples/ex7.csv')
reader = csv.reader(f)

In [45]:
# 对这个reader进行迭代会为每行产生一个元组(并移除了所有的引号)
for line in reader:
    print(line)

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3']


In [46]:
# 读取文件到一个多行列表
with open('examples/ex7.csv') as f:
    lines = list(csv.reader(f))

In [47]:
# 分为标题行和数据行
header, values = lines[0], lines[1:]

In [48]:
# 用字典构造式和zip(*values) (将行转置为列，创建数据列的字典)
data_dict = {h: v for h, v in zip(header, zip(*values))}

In [49]:
data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

In [50]:
# CSV文件有多种风格
# 可以定义csv.Dialect的一个子类即可定义出新格式
class my_dialect(csv.Dialect):
    lineterminator = '\n'
    delimiter = ';'
    quotechar = '"'
    quoting = csv.QUOTE_MINIMAL

In [52]:
f = open('examples/ex7.csv')
reader = csv.reader(f, dialect=my_dialect)

In [53]:
# 也可以不必定义子类，直接将CSV方言参数传入csv.reader的关键字参数
reader = csv.reader(f, delimiter='|')

In [54]:
# 对于更复杂或固定的多字符分隔符文件，将无法使用csv模块
# 这种情况下， 只能使用字符串的split方法或正则表达式方法re.split进行拆分清理工作
# 需要手动写入被分隔的文件时，你可以使用csv.writer

with open('examples/mydata.csv', 'w') as f:
    writer = csv.writer(f, dialect=my_dialect)
    writer.writerow(('one', 'two', 'three')) # 里面要加括号，元组
    writer.writerow(('1', '2', '3'))
    writer.writerow(('4', '5', '6'))  
    writer.writerow(('7', '8', '9'))

## JSON数据

In [55]:
obj = """
{"name": "Wes",
"places_lived": ["United States", "Spain", "Germany"],
"pet": null,
"siblings": [{"name": "Scott", "age": 30, "pets":["Zeus", "Zuko"]},
             {"name": "Katie", "age": 38, "pets":["Sixes", "Stache", "Cisco"]}]
}
"""

In [56]:
import json
res = json.loads(obj) # 转换成Python形式

In [57]:
res

{'name': 'Wes',
 'places_lived': ['United States', 'Spain', 'Germany'],
 'pet': None,
 'siblings': [{'name': 'Scott', 'age': 30, 'pets': ['Zeus', 'Zuko']},
  {'name': 'Katie', 'age': 38, 'pets': ['Sixes', 'Stache', 'Cisco']}]}

In [58]:
# json.dumps将Python对象转换成JSON格式
asjson = json.dumps(res)

In [59]:
# 转换成DataFrame
siblings = pd.DataFrame(res['siblings'], columns=['name', 'age'])

In [60]:
siblings

,name,age
0,Scott,30
1,Katie,38


In [61]:
# pandas.read_json可以自动将特别格式的JSON数据集转换为Series或DataFrame
!type examples\example.json

Active code page: 65001
[{"a": 1, "b": 2, "c": 3},
 {"a": 4, "b": 5, "c": 6},
 {"a": 7, "b": 8, "c": 9}]


In [62]:
data = pd.read_json('examples/example.json')

In [63]:
data

,a,b,c
0,1,2,3
1,4,5,6
2,7,8,9


In [64]:
# 将数据从pandas输出到JSON
print(data.to_json())

{"a":{"0":1,"1":4,"2":7},"b":{"0":2,"1":5,"2":8},"c":{"0":3,"1":6,"2":9}}


In [65]:
print(data.to_json(orient='records')) # 以记录形式显示

[{"a":1,"b":2,"c":3},{"a":4,"b":5,"c":6},{"a":7,"b":8,"c":9}]


## XML和HTML：Web信息收集

In [66]:
tables = pd.read_html('examples/fdic_failed_bank_list.html')

In [67]:
len(tables)

1

In [68]:
failures = tables[0]

In [69]:
failures.head()

,Bank Name,City,ST,CERT,Acquiring Institution,Closing Date,Updated Date
0,Allied Bank,Mulberry,AR,91,Today's Bank,"September 23, 2016","November 17, 2016"
1,The Woodbury Banking Company,Woodbury,GA,11297,United Bank,"August 19, 2016","November 17, 2016"
2,First CornerStone Bank,King of Prussia,PA,35312,First-Citizens Bank & Trust Company,"May 6, 2016","September 6, 2016"
3,Trust Company Bank,Memphis,TN,9956,The Bank of Fayette County,"April 29, 2016","September 6, 2016"
4,North Milwaukee State Bank,Milwaukee,WI,20364,First-Citizens Bank & Trust Company,"March 11, 2016","June 16, 2016"


In [70]:
close_timestamps = pd.to_datetime(failures['Closing Date'])

In [71]:
close_timestamps.dt.year.value_counts()

2010    157
2009    140
2011     92
2012     51
2008     25
       ... 
2004      4
2001      4
2007      3
2003      3
2000      2
Name: Closing Date, Length: 15, dtype: int64

## 利用lxml.objectify解析XML

In [72]:
from lxml import objectify
path = 'datasets/mta_perf/Performance_MNR.xml'

In [73]:
parsed = objectify.parse(open(path))

In [74]:
root = parsed.getroot()

In [75]:
data = []

In [76]:
# root.INDICATOR返回一个生成器，可以产生每一个<INDICATOR>XML元素。对于每条记录，我们可以
# 将标签名称的字典（如YTD_ACTUAL）填充为数据值（不包括几个标签）
skip_fields = ['PARENT_SEQ', 'INDICATOR_SEQ', 'DESIRED_CHANGE', 'DECIMAL_PLACES']

for elt in root.INDICATOR:
    el_data = {}
    for child in elt.getchildren():
        if child.tag in skip_fields:
            continue
        el_data[child.tag] = child.pyval
    data.append(el_data)

In [77]:
perf = pd.DataFrame(data)

In [78]:
perf.head()

,AGENCY_NAME,CATEGORY,DESCRIPTION,FREQUENCY,INDICATOR_NAME,INDICATOR_UNIT,MONTHLY_ACTUAL,MONTHLY_TARGET,PERIOD_MONTH,PERIOD_YEAR,YTD_ACTUAL,YTD_TARGET
0,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96.9,95,1,2008,96.9,95
1,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,95,95,2,2008,96,95
2,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96.9,95,3,2008,96.3,95
3,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,98.3,95,4,2008,96.8,95
4,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,95.8,95,5,2008,96.6,95


In [79]:
from io import StringIO
tag = '<a href="http://www.google.com">Google</a>'
root = objectify.parse(StringIO(tag)).getroot()

In [80]:
root

<Element a at 0x1d7c7c84d48>

In [81]:
root.get('href')

'http://www.google.com'

In [82]:
root.text

'Google'

# 二进制格式

In [83]:
# pandas 内建的pickle序列化模块进行二进制格式操作是存储数据最高效、最方便的方式之一。
frame = pd.read_csv('examples/ex1.csv')
frame

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [84]:
frame.to_pickle('examples/fram_pickle')

In [85]:
pd.read_pickle('examples/fram_pickle')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [86]:
# pandas 内建支持HDF5和MessagePack格式

## 使用HDF5格式

In [87]:
frame = pd.DataFrame({'a': np.random.randn(100)})
store = pd.HDFStore('mydata.h5')
store['obj1'] = frame
store['obj1_col'] = frame['a']
store

<class 'pandas.io.pytables.HDFStore'>
File path: mydata.h5

In [88]:
store['obj1']

,a
0,-0.843469
1,-0.769408
2,-1.274294
3,1.644214
4,-1.587970
...,...
95,-1.011693
96,-0.049810
97,-0.205606
98,-0.303773


In [89]:
# put是store ['obj2']=frame方法的显式版本，但允许我们设置其他选项，如存储格式
store.put('obj2', frame, format='table')
store.select('obj2', where=['index >= 10 and index <= 15'])
# store['obj1'] 未设置为table类型

,a
10,-2.329332
11,-0.398105
12,1.013429
13,-0.395434
14,-0.840550
15,1.631229


In [90]:
frame.to_hdf('examples/mydata.h5','obj3',format='table')

In [91]:
pd.read_hdf('examples/mydata.h5','obj3',where=['index < 5'])

,a
0,-0.843469
1,-0.769408
2,-1.274294
3,1.644214
4,-1.587970


In [92]:
store.select('obj3', where=['index >= 10 and index <= 15'])

,a
10,-1.641191
11,-1.114780
12,-0.782826
13,-1.501938
14,0.003994
15,-0.430097


In [93]:
# HDF5并不是数据库，它是一种适合一次写入多次读取的数据集。
# 尽管数据可以在任何时间添加到文件中，但如果多个写入者持续写入，文件可能会损坏

## 读取Microsoft Excel文件

In [94]:
xlsx = pd.ExcelFile('examples/ex1.xlsx')

In [95]:
pd.read_excel(xlsx, 'Sheet1')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [96]:
type(xlsx)

pandas.io.excel.ExcelFile

In [97]:
frame = pd.read_excel('examples/ex1.xlsx', 'Sheet1')
frame

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [98]:
# 写入到excel
writer = pd.ExcelWriter('examples/ex2.xlsx')
frame.to_excel(writer, 'Sheet1')
writer.save()

In [99]:
type(writer)

pandas.io.excel._XlsxWriter

In [100]:
# 直接将文件名传给to_excel,避免直接调用ExcelWriter
frame.to_excel('examples/ex2.xlsx')

# 与数据库交互

In [101]:
import sqlite3
query = """
CREATE TABLE test(
a VARCHAR(20), 
b VARCHAR(20),
c REAL,
d INTEGER
);
"""

In [102]:
con = sqlite3.connect('examples/mydata.sqplite')

In [117]:
con.execute(query)

In [104]:
con.commit()

In [105]:
data = [('Atlanta', 'Georgia', 1.25, 6),
        ('Tallahassee', 'Florida', 2.6, 3),
        ('Sacramento', 'California', 1.7, 5)]

In [106]:
stmt = "INSERT INTO test VALUES(? , ? , ? , ?)"

In [107]:
con.executemany(stmt, data)

In [108]:
con.commit()

In [109]:
cursor = con.execute('select * from test')

In [110]:
rows = cursor.fetchall()

In [111]:
rows

[('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5),
 ('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5)]

In [112]:
cursor.description

(('a', None, None, None, None, None, None),
 ('b', None, None, None, None, None, None),
 ('c', None, None, None, None, None, None),
 ('d', None, None, None, None, None, None))

In [113]:
pd.DataFrame(rows, columns=[x[0] for x in cursor.description])

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5
3,Atlanta,Georgia,1.25,6
4,Tallahassee,Florida,2.60,3
5,Sacramento,California,1.70,5


In [114]:
import sqlalchemy as sqla

In [115]:
db = sqla.create_engine('sqlite:///examples/mydata.sqplite')

In [116]:
pd.read_sql('select * from test', db)

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5
3,Atlanta,Georgia,1.25,6
4,Tallahassee,Florida,2.60,3
5,Sacramento,California,1.70,5
